Заказчик проведения данной работы - отдел маркетинга.  

Информация нужна менеджеру, который занимается маркетинговыми компаниями и работает в тесной связке с менеджерами продуктов.   

На выходе хотят получить сегменты пользователей по продуктам, который им подходят, либо сигменты по количеству используемых продуктов и признакам отточности и неотточности. 

Банк хочет переработать продуктовую линейку, чтобы удержать отточных клиентов, а также привлечь лояльных банку клиентов.   

In [1]:
# Загружаем библиотеки
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math as mth
import seaborn as sns
import warnings; warnings.filterwarnings(action='once')
import scipy.stats as st
import datetime as dt
import plotly.express as px
import plotly.subplots as sp
from plotly import graph_objects as go
from collections import Counter
from pandas.plotting import register_matplotlib_converters

In [2]:
# Указываем базовые настройки 
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
register_matplotlib_converters()

In [3]:
# Указываем настройки графиков
large = 16; med = 10; small = 8
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (10, 6),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)
sns.set_style("whitegrid")
%matplotlib inline

In [4]:
# Указываем путь к файлу
try:
    data_raw = pd.read_csv('C:/Users/Boris/Desktop/datasets/bank_scrooge.csv')
except: 
    data_raw = pd.read_csv('/Users/borisdarzania/Desktop/datasets/bank_scrooge.csv')

In [5]:
# Выведем первые 5 строчек набора данных
data_raw.head()

,USERID,score,city,gender,age,equity,balance,products,credit_card,last_activity,EST_SALARY,churn
0,183012,850.00,Рыбинск,Ж,25.00,1,59214.82,2,0,1,75719.14,1
1,146556,861.00,Рыбинск,Ж,37.00,5,850594.33,3,1,0,86621.77,0
2,120722,892.00,Рыбинск,Ж,30.00,0,NaN,1,1,1,107683.34,0
3,225363,866.00,Ярославль,Ж,51.00,5,1524746.26,2,0,1,174423.53,1
4,157978,730.00,Ярославль,М,34.00,5,174.00,1,1,0,67353.16,1


In [6]:
# Посмотрим информацию по данным в таблице
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   USERID         10000 non-null  int64  
 1   score          10000 non-null  float64
 2   city           10000 non-null  object 
 3   gender         10000 non-null  object 
 4   age            9974 non-null   float64
 5   equity         10000 non-null  int64  
 6   balance        7705 non-null   float64
 7   products       10000 non-null  int64  
 8   credit_card    10000 non-null  int64  
 9   last_activity  10000 non-null  int64  
 10  EST_SALARY     10000 non-null  float64
 11  churn          10000 non-null  int64  
dtypes: float64(4), int64(6), object(2)
memory usage: 937.6+ KB


Описание данных:  
**`USERID`** — идентификатор пользователя,  
**`score`** — баллы кредитного скоринга,  
**`city`** — город,  
**`gender`** — пол,  
**`age`** — возраст,  
**`equity`** — количество баллов собственности,  
**`balance`** — баланс на счёте,  
**`products`** — количество продуктов, которыми пользуется клиент,  
**`credit_card`** — есть ли кредитная карта,  
**`last_activity`** — активный клиент,  
**`EST_SALARY`** — оценочный доход клиента,  
**`сhurn`** — признак оттока.  

Нам известно, что столбцы **score**, **equality** и **est_salary** являются внешними для банка. Т.е. получаются из внешних систем и являются композитной оценкой. 

In [7]:
data_raw.describe()

,USERID,score,age,equity,balance,products,credit_card,last_activity,EST_SALARY,churn
count,10000.00,10000.00,9974.00,10000.00,7705.00,10000.00,10000.00,10000.00,10000.00,10000.00
mean,171814.71,848.70,42.73,2.63,827794.31,1.87,0.68,0.52,147866.89,0.18
std,33708.24,65.45,12.18,1.98,1980614.15,0.79,0.47,0.50,139388.51,0.39
min,94561.00,642.00,18.00,0.00,0.00,0.00,0.00,0.00,2546.30,0.00
25%,142810.25,802.00,33.00,0.00,295554.16,1.00,0.00,0.00,75251.90,0.00
50%,172728.00,853.00,40.00,3.00,524272.20,2.00,1.00,1.00,119658.10,0.00
75%,201261.75,900.00,51.00,4.00,980705.85,2.00,1.00,1.00,174500.54,0.00
max,229145.00,1000.00,86.00,9.00,119113552.01,5.00,1.00,1.00,1395064.45,1.00


## Предварительная обработка данных

In [8]:
# Зададим вспомогательный словарь
thesaurus = {
    'user_id' : 'Идентификатор',
    'score' : 'Баллы скоринга',
    'city' : 'Город',
    'gender' : 'Пол',
    'age' : 'Возраст',
    'equity' : 'Баллы собственности',
    'balance' : 'Баланс',
    'products' : 'Продукты',
    'credit_card' : 'Кредитная карта',
    'last_activity' : 'Активность',
    'est_salary' : 'Доход',
    'сhurn' : 'Отток'
}

In [9]:
# Переименуем столбцы
data = data_raw.copy()
data = data.rename(columns={'USERID':'user_id'})
data.columns = data.columns.str.lower()

In [10]:
# Зададим функцию с параметрами необходимого набора данных
def first_look(df):
    
    # Сделаем новый набор данных, в который будет добавлять искомые значения
    fl_df = pd.DataFrame(columns=list(df.columns))
    
    # Начнем итерировать по колонкам набора данных
    for column in df.columns:
        fl_df.loc[1, column] = df[column].nunique()
        fl_df.loc[2, column] = df[column].isna().sum()
        fl_df.loc[3, column] = round(df[column].isna().sum() / len(df[column]), 4) * 100
        fl_df.loc[4, column] = df[column].duplicated().sum()
        fl_df.loc[5, column] = round(df[column].duplicated().sum() / len(df[column]), 4) * 100
    
    # Переименуем индексы строк
    fl_df.index = ['Уникальные значения',
                   'Пропуски',
                   'Пропуски, %',
                   'Дубликаты',
                   'Дубликаты, %']
    
    fl_df = fl_df.rename(columns = dict(thesaurus))
    fl_df = fl_df.fillna('')
    
    # Выводим получившийся набор данных
    return display(fl_df)

In [11]:
first_look(data)

,Идентификатор,Баллы скоринга,Город,Пол,Возраст,Баллы собственности,Баланс,Продукты,Кредитная карта,Активность,Доход,churn
Уникальные значения,9927.00,348.00,3.00,2.00,68.00,10.00,7701.00,6.00,2.00,2.00,9996.00,2.00
Пропуски,0.00,0.00,0.00,0.00,26.00,0.00,2295.00,0.00,0.00,0.00,0.00,0.00
"Пропуски, %",0.00,0.00,0.00,0.00,0.26,0.00,22.95,0.00,0.00,0.00,0.00,0.00
Дубликаты,73.00,9652.00,9997.00,9998.00,9931.00,9990.00,2298.00,9994.00,9998.00,9998.00,4.00,9998.00
"Дубликаты, %",0.73,96.52,99.97,99.98,99.31,99.90,22.98,99.94,99.98,99.98,0.04,99.98


Получили первый осмотр набор данных, приступим к дальнейшей обработке:

In [12]:
# Посмотрим поближе дубликаты в столбце user_id
data[data['user_id'].duplicated(keep=False)].sort_values(by='user_id')

,user_id,score,city,gender,age,equity,balance,products,credit_card,last_activity,est_salary,churn
1893,116540,883.00,Рыбинск,Ж,55.00,1,362756.49,3,0,1,175920.48,1
7694,116540,887.00,Ярославль,Ж,38.00,0,NaN,1,0,1,119247.61,0
7542,117943,880.00,Ярославль,Ж,40.00,0,NaN,1,1,0,137718.93,0
4866,117943,855.00,Рыбинск,Ж,32.00,6,1036832.93,4,1,1,107792.71,1
5896,120258,905.00,Ярославль,М,30.00,0,NaN,1,1,1,146427.96,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2597,226719,990.00,Ярославль,М,37.00,4,14648692.14,2,0,0,934412.61,1
8205,227795,840.00,Рыбинск,М,34.00,2,350768.03,1,1,0,102036.14,1
8497,227795,839.00,Ярославль,М,34.00,2,326593.14,2,1,0,103314.92,0
6457,228075,839.00,Рыбинск,М,39.00,5,507199.85,3,0,1,85195.80,0


Как мы видим, это не дубликаты, а просто один идентификатор был присвоен разым людям. Таким образом, мы не будет удалять данные строки.

In [13]:
# Удаляем пропуски в столбце age
data = data.dropna(subset=['age'])

In [15]:
data[data['balance'].isna()]

,user_id,score,city,gender,age,equity,balance,products,credit_card,last_activity,est_salary,churn
2,120722,892.00,Рыбинск,Ж,30.00,0,NaN,1,1,1,107683.34,0
9,133130,906.00,Ярославль,Ж,67.00,0,NaN,1,0,1,238055.53,0
10,148929,927.00,Ростов,М,52.00,0,NaN,1,1,1,196820.07,0
11,172184,921.00,Ростов,М,41.00,0,NaN,1,1,1,217469.48,0
19,127034,922.00,Рыбинск,Ж,53.00,0,NaN,1,0,0,147094.82,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9976,208085,876.00,Ростов,М,38.00,0,NaN,1,0,0,171763.69,0
9984,125941,729.00,Ярославль,Ж,42.00,0,NaN,1,1,1,687538.70,0
9993,219924,884.00,Рыбинск,Ж,36.00,0,NaN,1,1,1,169844.88,0
9996,139170,894.00,Ярославль,М,46.00,0,NaN,1,1,0,196898.29,0


In [16]:
data['balance'] = data['balance'].fillna(0)

In [17]:
data.isna().sum()

user_id          0
score            0
city             0
gender           0
age              0
equity           0
balance          0
products         0
credit_card      0
last_activity    0
est_salary       0
churn            0
dtype: int64

In [18]:
data['products'].unique()

array([2, 3, 1, 4, 5, 0])

In [19]:
data[data['products']==0]

,user_id,score,city,gender,age,equity,balance,products,credit_card,last_activity,est_salary,churn
8957,147837,962.00,Рыбинск,Ж,79.00,3,0.00,0,0,0,25063.96,1
